In [ ]:
import google.generativeai as genai
import base64
from PIL import Image
from io import BytesIO
import requests
import os
import json
import re
import pandas as pd
from vertexai.preview.generative_models import Part
                

In [ ]:

genai.configure(api_key="")


In [ ]:
model = genai.GenerativeModel("gemini-2.0-flash-lite")


In [ ]:
def encode_image(path):
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def escape_inches(st):
    # Add a single backslash before a " that follows a digit
    return re.sub(r'(?<=\d)"', '\\"', st)


In [ ]:
img_path = '../Tell2Design Data/General Data/floorplan_image/'
save_path = '../Tell2Design Data/topological_ordering/'
json_path = '../Tell2Design Data/t2d.json'

In [ ]:
with open(json_path) as f:
    data = json.load(f)

In [ ]:
data['77414.png']

In [ ]:
system_prompt = {
    "role": "user",
    "parts": [
        {
            "text": """You are a top-level architect with years of experience reading detailed architectural site plans.

            <color-scheme>
            Use this exact color-to-room mapping:
            - balcony:      (107, 142, 35)
            - common room:  (255, 215, 0)
            - bathroom:     (173, 216, 230)
            - kitchen:      (240, 128, 128)
            - master room:  (255, 165, 0)
            - living room:  (238, 232, 170)
            - entrance:     (255, 0, 0)
            - dining room:  (218, 112, 214)
            - storage:      (221, 160, 221)
            </color-scheme>

            <assumptions>
            - The top of the image is North.
            - Image size is 256×256 pixels.
            - If multiple rooms share a type, label them type_1, type_2, … (e.g., common_1, common_2).
            </assumptions>
            
            <task>
            1. Provide a step-by-step reasoning process for producing a topological ordering of rooms from north to south, breaking ties west to east.
            </task>
            
           
            Strictly follow this output format
            No json format output
            <output format>
            "CoT:" ...
            "answer:" ...
            </output format>
            
            
            """
            
        }
    ]
}


In [ ]:
def save_pred(output, path, dp):
    output = output.replace('*', "")

    reason_start = re.search(r'CoT:', output, re.DOTALL).span()[0]
    reason_end = re.search(r'CoT:', output, re.DOTALL).span()[1]
    answer_start = re.search(r'answer:', output, re.DOTALL).span()[0]
    answer_end = re.search(r'answer:', output, re.DOTALL).span()[1]

    
    reason_text = output[reason_end:answer_start]
    reason_lines = [line.strip() for line in reason_text.strip().split("\n") if line.strip()]
    answer_text = output[answer_end:]
    answer_lines = [line.strip() for line in answer_text.strip().split("\n") if line.strip()]

    result = {
            "Reason": reason_lines,
            "Answer": answer_lines,
            "gt_desc": dp['desc'],
            "num_rooms": dp['num_rooms'],
            "rooms":dp['rooms']
            
        }
    with open(path, "w") as f:
        json.dump(result, f, indent=2)
     

In [ ]:
for i, each in enumerate(data.keys()):
    print(i)
    if i < 227:
        continue
    
    img = img_path + each
    messages = [system_prompt]
    test_img = encode_image(img)
    messages.append({
    "role": "user",
    "parts": [
        {"mime_type": "image/jpeg", "data": test_img},
        {"text": "Give chain of thoughts and answer the questions for this floor plan."}
        ]
    })
    
    response = model.generate_content(messages)
    output = response.text
    path = save_path + each.split('.png')[0] + '.json'
    save_pred(output, path, data[each])
    
   

In [ ]:
response

In [ ]:
output = output.replace('*', "")

reason_start = re.search(r'CoT:', output, re.DOTALL).span()[0]
reason_end = re.search(r'CoT:', output, re.DOTALL).span()[1]
answer_start = re.search(r'answer:', output, re.DOTALL).span()[0]
answer_end = re.search(r'answer:', output, re.DOTALL).span()[1]


reason_text = output[reason_end:answer_start]
reason_lines = [line.strip() for line in reason_text.strip().split("\n") if line.strip()]
answer_text = output[answer_end:]
answer_lines = [line.strip() for line in answer_text.strip().split("\n") if line.strip()]
